###by J.Lee ###

This is simple notebook file that creates animated brain image.

'alpha_composit' method in pillow's Image class is used.

Truth label and prediction label are superimposed on the mri image.

'read_img_sitk()' funciton is derived from Naomi Fridman's repository.

Flair and t1ce nii images are used to create baseimage.

In [1]:
import SimpleITK as sitk
import numpy as np

def read_img_sitk(img):
    '''
    reads nii image and returns a numpy array.
    '''
    inputImage = sitk.ReadImage(img)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    image = sitk.GetArrayFromImage(inputImage)
    return image

def nii_to_8bit_npy(img_dirs):
    """
    recieves a list of image_diriectories e.g.[C:/ab/cd.nii, C:/ab/ce.nii, C:/de/fg.nii]
    returns a merged 8bit-scale(i.e.0-255) ndarray.
    """
    image = 0.
    for i in img_dirs:
        image += read_img_sitk(i)
    image = (image/np.max(image))*255
    return image//1

In [2]:
directory = 'C:/IAMEDIC/Jaeho_code/data/preprocessed_N4/HGG/BraTS19_CBICA_AAB_1/'
img_dirs = [directory+'flair.nii.gz', directory+'t1ce.nii.gz']
pred_dir = 'C:/IAMEDIC/Jaeho_code/demo_task1/saves/training_to_upload_1.3/BraTS19_CBICA_AAB_1.nii.gz'
true_dir = directory+'truth.nii.gz'

## MRI Image preprocessing

In [8]:
mri_img = nii_to_8bit_npy(img_dirs)

#turning to gray-color,un-transparent RGBA image
mri_img = np.stack((mri_img, mri_img, mri_img, np.ones(mri_img.shape)*255), axis = -1).astype('uint8')

## Segmentation Image Processing

In [9]:
#1. Image Preparation

seg_pred = read_img_sitk(pred_dir)
seg_true = read_img_sitk(true_dir)

#Edema:2, NEC:4, ET:1
#pred_images
pred_ED = np.where(seg_pred==2, 1, 0 ).astype('uint8') #2
pred_NC = np.where(seg_pred==4, 1, 0 ).astype('uint8') #4
pred_ET = np.where(seg_pred==1, 1, 0 ).astype('uint8') #1
pred_TC = pred_NC+pred_ET #1+4
pred_WT = pred_ED+pred_TC #1+2+4

#true_images
true_ED = np.where(seg_true==2, 1, 0 ).astype('uint8')
true_NC = np.where(seg_true==4, 1, 0 ).astype('uint8')
true_ET = np.where(seg_true==1, 1, 0 ).astype('uint8')
true_TC = true_NC+true_ET
true_WT = true_ED+true_TC

# pred_WT = np.stack((pred_WT,)*3, axis=-1)
# true_WT = np.stack((true_WT,)*3, axis=-1)

#turning to RGBA image: pred = red, true = blue
pred_ED = np.stack((pred_ED*255,pred_ED*0,pred_ED*0,pred_ED*128), axis = -1)
pred_NC = np.stack((pred_NC*255,pred_NC*0,pred_NC*0,pred_NC*128), axis = -1)
pred_ET = np.stack((pred_ET*255,pred_ET*0,pred_ET*0,pred_ET*128), axis = -1) 
pred_TC = np.stack((pred_TC*255,pred_TC*0,pred_TC*0,pred_TC*128), axis = -1) 
pred_WT = np.stack((pred_WT*255,pred_WT*0,pred_WT*0,pred_WT*128), axis = -1) 

true_ED = np.stack((true_ED*0,true_ED*0,true_ED*255,true_ED*128), axis = -1)
true_NC = np.stack((true_NC*0,true_NC*0,true_NC*255,true_NC*128), axis = -1)
true_ET = np.stack((true_ET*0,true_ET*0,true_ET*255,true_ET*128), axis = -1) 
true_TC = np.stack((true_TC*0,true_TC*0,true_TC*255,true_TC*128), axis = -1) 
true_WT = np.stack((true_WT*0,true_WT*0,true_WT*255,true_WT*128), axis = -1) 

print(pred_WT.shape)
print(true_WT.shape)

(155, 240, 240, 4)
(155, 240, 240, 4)


## Overlaying (Super-composing)

In [30]:
from PIL import Image

def alpha_composit_3D(img1, img2, img3):
    """
    receives two same shaped 3D-RGBA arrays. e.g.(155, 240, 240, 4)
    returns a list of 2d alpha composited images sliced by axis0.
    """
    img1_arr = [Image.fromarray(i) for i in img1]
    img2_arr = [Image.fromarray(i) for i in img2]
    alpha_3D = []
    for i,j in zip(img1_arr,img2_arr):
        alpha_3D.append(Image.alpha_composite(i,j))    

    return alpha3D

def alpha_composit_3D_3(img1, img2, img3):
    """
    receives three same shaped 3D-RGBA arrays. e.g.(155, 240, 240, 4)
    returns a list of 2d alpha composited images sliced by axis0.
    first two images will be composited and the third one will be composited on top.
    """
    img1_arr = [Image.fromarray(i) for i in img1]
    img2_arr = [Image.fromarray(i) for i in img2]
    alpha_3D = []
    for i,j in zip(img1_arr,img2_arr):
        alpha_3D.append(Image.alpha_composite(i,j))    

    alpha3D_2 = []
    img3_arr = [Image.fromarray(i) for i in img3]
    for i,j in zip(alpha_3D, img3_arr):
        alpha3D_2.append(Image.alpha_composite(i,j))
    return alpha3D_2
    

def create_gif_from_images(images, path = 'img.gif', duration = 100):
    images[0].save(path,
                   save_all=True,
                   append_images=images[1:],
                   optimize=False,
                   duration=duration,
                   loop=0)
    print(f"{path} has been created successfully.")
    return 

In [31]:
WT_overlay = alpha_composit_3D_3(mri_img[45:125,:,:,:], true_WT[45:125,:,:,:], pred_WT[45:125,:,:,:])
WT_overlay[50].show()
create_gif_from_images(WT_overlay, 'WT.gif')

WT.gif has been created successfully.


In [32]:
TC_overlay = alpha_composit_3D_3(mri_img[45:125,:,:,:], true_TC[45:125,:,:,:], pred_TC[45:125,:,:,:])
TC_overlay[50].show()
create_gif_from_images(TC_overlay, 'TC.gif')

TC.gif has been created successfully.


In [38]:
ET_overlay = alpha_composit_3D_3(mri_img[45:125,:,:,:], true_ET[45:125,:,:,:], pred_ET[45:125,:,:,:])
ET_overlay[50].show()
create_gif_from_images(TC_overlay, 'ET.gif')

ET.gif has been created successfully.


In [39]:
create_gif_from_images([Image.fromarray(i) for i in mri_img[45:125,:,:,:]], 'MRI(T1ce_Flair).gif')

MRI(T1ce_Flair).gif has been created successfully.


In [11]:
# from PIL import Image
# pred_ED = Image.fromarray(pred_ED[80,:,:,:])
# pred_NC = Image.fromarray(pred_NC[80,:,:,:])
# pred_ET = Image.fromarray(pred_ET[80,:,:,:])
# pred_TC = Image.fromarray(pred_TC[80,:,:,:])
# pred_WT = Image.fromarray(pred_WT[80,:,:,:])

# true_ED = Image.fromarray(true_ED[80,:,:,:])
# true_NC = Image.fromarray(true_NC[80,:,:,:])
# true_ET = Image.fromarray(true_ET[80,:,:,:])
# true_TC = Image.fromarray(true_TC[80,:,:,:])
# true_WT = Image.fromarray(true_WT[80,:,:,:])

# pred_ED.mode

'RGBA'

In [14]:
# true_WT.show()
# pred_WT.show()
# x = Image.alpha_composite(true_WT, pred_WT)
# x.show()
# y = Image.alpha_composite(x, pred_TC)
# y.show()
# y.mode

In [44]:
TC_overlay[25].show()

In [45]:
ET_overlay[30].show()

In [46]:
WT_overlay[30].show()